In [10]:
import pandas as pd
import numpy as np

def calculate_yield(yield_interest = 8, starting_fyt_interest = 4,
                    ending_fyt_interest = 8, step_size = .001,
                    input_amount = 1000, liquidity = 50, gas_fee = .0019,
                    days_maturity = 90, days_matured = 0, target_interest = 0,
                    time_stretch = 15):
  columns = (
    'FYT Interest',
    'Actual Interest With Slippage',
    'Total Expenditure',
    'Received at Maturity',
    'APY'
  )

  values = []
  current_interest = ending_fyt_interest

  # zero out accumulated interest
  accumulated_interest = 0

  # Include time stretch
  t = (days_maturity - days_matured) / (365 * time_stretch)

  for i in reversed(range (0, int(((ending_fyt_interest - starting_fyt_interest)/step_size)) + 1)):
    if days_matured > 0:
      accumulated_interest = input_amount * days_matured/365 * yield_interest/100

    # If the apy is 10% and there is 6 months remaining, the applied interest is 5%
    applied_interest = (days_maturity - days_matured) / 365 * current_interest

    y_start = liquidity
    x_start = y_start/((1-applied_interest/100)**(-1/t)-1)
    total_supply = x_start

    k = (y_start + total_supply)**(1-t) + x_start**(1-t)
    j = input_amount - accumulated_interest
    i = x_start - pow(k-pow(y_start + total_supply + j,1-t),1/(1-t))
    fee = (j - i) * .1
    i = i - fee

    fyt_discount_with_slippage = (1 - (i / j)) * 100

    row = []
    row.append(input_amount)
    row.append(current_interest)
    row.append(round(fyt_discount_with_slippage * (365/ (days_maturity - days_matured)), 2))
    fyt_expenditure = input_amount - accumulated_interest - i
    expenditure = fyt_expenditure + gas_fee + accumulated_interest

    received = (input_amount * yield_interest / 100) * (days_maturity) / 365
    row.append(expenditure)
    row.append(received)
    row.append(received - expenditure)
    apy = round(((received / expenditure) - 1) * 365 / (days_maturity - days_matured) * 100, 2)
    row.append(apy)
    if ((received - expenditure) * 10) > (target_interest * input_amount * (days_maturity - days_matured) / 365 / 100):
      return row

    current_interest = current_interest - step_size


In [20]:
import ipywidgets as widgets

def button_run_on_click(_):
    status_label.value = "running...."

    columns = ['Input', 'FYT APY', 'Slippage APY', 'Spent', 'Received', 'Gain', 'APY']
    values = []

    for i in np.arange(.5, 5, .1):
      results = calculate_yield(yield_interest=yield_interest_input.value, \
                                input_amount = i, \
                                liquidity = liquidity_input.value, \
                                time_stretch = time_stretch_input.value, \
                                target_interest=target_interest_input.value)
      if results is not None:
        values.append(results)

    df = pd.DataFrame(values, columns = columns)
    pd.options.display.width = 20
    df.style.set_properties(**{'text-align': 'left'})
    
    status_label.value = ""

    result_box = setup_ui(df)

    main_box.children = [yield_interest_input, target_interest_input, liquidity_input, time_stretch_input, button_run, status_label, result_box]


def setup_ui(df):

    out = widgets.Output()
    with out:
        display(df)
    return out

status_label = widgets.Label()
status_label.layout.width = '300px'

yield_interest_input = widgets.FloatText(description='speculated interest', value=8)
target_interest_input = widgets.FloatText(description='target interest', value=16)
liquidity_input = widgets.FloatText(description='liquidity', value=50)
time_stretch_input = widgets.FloatText(description='time stretch', value=15)
button_run = widgets.Button(description="Run")
main_box = widgets.VBox([yield_interest_input, target_interest_input, liquidity_input, time_stretch_input, button_run, status_label])
button_run.on_click(button_run_on_click)
display(main_box)
